In [1]:
import json
import os
import random
import numpy as np
from utils import smiles_utils
from utils import polysmiles
import hoomd
import mbuild as mb
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import foyer
from foyer import Forcefield
import py3Dmol
import ele

In [2]:
test_system = System(molecule="PEKK", para_weight=0.60,
                    density=1.30, n_compounds=[20,20], polymer_lengths=[5, 10])
packed_system = test_system.pack()
packed_system.visualize()

/home/chris/miniconda3/envs/uli/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'System' is not defined

In [ ]:
class System():
    def __init__(self,
                 molecule,
                 para_weight,
                 density,
                 n_compounds,
                 polymer_lengths,
                 forcefield=None,
                 pdi=None,
                 M_n=None,
                 remove_hydrogens=False
            ):
        
        self.molecule = molecule
        self.para_weight = para_weight
        self.density = density
        self.remove_hydrogens = remove_hydrogens
        self.pdi = pdi
        self.system_mass = 0
        
        if self.pdi:
            pass
            '''
            Here, call a function that samples from some distribution
            pass in pdi, n_compounds, M_n?
            self.polymer_lengths and self.n_compounds defined from that function
            '''
        else: # Do some validation, get things in the correct data types
            if not isinstance(n_compounds, list):
                self.n_compounds = [n_compounds]
            else:
                self.n_compounds = n_compounds

            if not isinstance(polymer_lengths, list):
                self.polymer_lengths = [polymer_lengths]
            else:
                self.polymer_lengths = polymer_lengths
        
        if len(self.n_compounds) != len(self.polymer_lengths):
            raise ValueError('n_compounds and polymer_lengths should be equal length')
    
    def pack(self, box_expand_factor=5):
        
        # Create compounds
        mb_compounds = []
        mb_n_compounds = []
        for _length, _n in zip(self.polymer_lengths, self.n_compounds):
            polymer = build_molecule(self.molecule, _length,
                                    self.para_weight)
            mb_compounds.append(polymer)
            mb_n_compounds.append(_n)
            mass = _n * np.sum(ele.element_from_symbol(p.name).mass for p in polymer.particles())
            self.system_mass += mass
        
        # Figure out correct box dimensions
        # Expand the box to make the PACKMOL step faster
        # Will shrink down to accurate L during simulation
        L = self._calculate_L() * box_expand_factor
        system = mb.packing.fill_box(
            mb_compounds,
            mb_n_compounds,
            box=[L, L, L],
            edge=0.5,
            fix_orientation=True)
        return system
    
    def _calculate_L(self):
        # Conversion from (amu/(g/cm^3)) to ang
        L = (self.system_mass / self.density) ** (1/3) * 1.841763
        L /= 10 # convert ang to nm
        return L
        

def build_molecule(molecule, length, para_weight):
    '''
    `build_molecule` uses SMILES strings to build up a polymer from monomers.
    The configuration of each monomer is determined by para_weight and the
    random_sequence() function.  
    Uses DeepSMILES behind the scenes to build up SMILES string for a polymer.
    
    Parameters
    ----------
    molecule : str
        The monomer molecule to be used to build up the polymer.
        Available options are limited  to the .json files in the compounds directory
        Use the molecule name as seen in the .json file without including .json
    length : int
        The number of monomer units in the final polymer molecule
    para_weight : float, limited to values between 0 and 1
        The relative amount of para configurations compared to meta.
        Passed into random_sequence() to determine the monomer sequence of the polymer.
        A 70/30 para to meta system would require a para_weight = 0.70
    
    Returns
    -------
    molecule_string_smiles : str
        The complete SMILES string of the polymer molecule
    '''
    f = open('compounds/{}.json'.format(molecule))
    mol_dict = json.load(f)    
    f.close()
    monomer_sequence = random_sequence(para_weight, length)
    molecule_string = '{}'

    for idx, config in enumerate(monomer_sequence):
        if idx == 0: # append template, but not brackets
            monomer_string = mol_dict['{}_template'.format(config)]
            molecule_string = molecule_string.format(monomer_string)
            if len(monomer_sequence) == 1:
                molecule_string = molecule_string.replace('{}', '')
                continue

        elif idx == length - 1: # Don't use template for last iteration
            brackets = polysmiles.count_brackets(mol_dict['{}_deep_smiles'.format(config)])
            monomer_string = mol_dict['{}_deep_smiles'.format(config)]
            molecule_string = molecule_string.format(monomer_string, brackets)

        else: # Continue using template plus brackets
            brackets = polysmiles.count_brackets(mol_dict['{}_deep_smiles'.format(config)])
            monomer_string = mol_dict['{}_template'.format(config)]
            molecule_string = molecule_string.format(monomer_string, brackets)
    
    molecule_string_smiles = smiles_utils.convert_smiles(deep = molecule_string)
    compound = mb.load(molecule_string_smiles, smiles=True)
    return compound

def random_sequence(para_weight=0.5, length=10):
    '''
    random_sequence returns a list containing a random sequence of strings 'para' and 'meta'.
    This is used by build_molecule() to create a complete SMILES string of a molecule.
    
    Parameters:
    -----------
    para_weight : float, limited to values between 0 and 1
        The relative amount of para configurations compared to meta.
        Defined in build_molecule()
    length : int
        The number of elements in the random sequence.
        Defined in build_molecule()
    '''
    meta_weight = 1 - para_weight
    options = ['para', 'meta']
    probability = [para_weight, meta_weight]
    sequence = random.choices(options, weights=probability, k=length)
    return sequence

In [ ]:
##############################################

In [ ]:
def build_system(packing='bcc'):
    '''
    Generate a simple LJ particle system using hoomd's create_lattice function
    packing 
    '''
    hoomd.context.initialize("")
    if packing == 'fcc':
        system = hoomd.init.create_lattice(unitcell=(hoomd.lattice.fcc(a=1.58)), n=6)
    elif packing == 'bcc':
        system = hoomd.init.create_lattice(unitcell=(hoomd.lattice.bcc(a=1.29)), n=6)
    hoomd.dump.gsd('{}_system.gsd'.format(packing), group = hoomd.group.all(), period=None, overwrite=True)
    return system

def hoomd_simulation(system, temp, tau):  
    nl = hoomd.md.nlist.cell()
    lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)
    lj.pair_coeff.set('A', 'A', alpha=1.0, epsilon=1.0, sigma=1.0)
    hoomd.md.integrate.mode_standard(dt=0.001)
    _all = hoomd.group.all()
    nvt = hoomd.md.integrate.nvt(group=_all, kT=temp, tau=tau)
    nvt.randomize_velocities(seed=23)
    hoomd.analyze.log(filename='{}-tau_out.log'.format(tau),
                      quantities=["time", "temperature", "potential_energy"],
                      period=100,
                      overwrite=True
                     )
    #hoomd.dump.gsd('tau-trajectory.gsd', period=5e3, group=_all, overwrite=True)
    hoomd.run(3e5)

# --------------------------------------------------------

PEEK:  
1 amu = 1.66054e-24 g  
1.32 g/cm^3  
1.32 g/nm^3  
C19H12O3  
monomer_amu = 288.302  

In [3]:
def build_peek_system(num_mols, poly_length, density):
    '''
    This function uses mBuild's packing functionality to create a very low dense system of molecules
    to allow for easier packing. A short simulation is then ran using HoomD to shrink the system to the
    desired starting density.
    '''
    peek_poly_smi = polysmiles.poly_smiles('occccC=O)ccccOcc*c*ccc6)))))))cc6)))))))cc6',
                                      length=poly_length)
    peek_poly = mb.load(peek_poly_smi, smiles=True)
    
    mol_amu = poly_length * 288.302
    mol_grams = mol_amu * 1.66054e-24
    system_mass = mol_grams * num_mols
    edge_cm = (system_mass / density)**(1/3)
    edge_nm = edge_cm * 1e7

    init_box = mb.Box([edge_nm*5]*3)
    system = mb.fill_box(peek_poly, num_mols, init_box)
    
    GAFF = foyer.forcefields.load_GAFF()
    system_pmd = GAFF.apply(system)
    create_hoomd_simulation(system_pmd, r_cut=1.2, auto_scale=True)
    forces = [f for f in hoomd.context.current.forces 
                if not isinstance(f, hoomd.md.charge.pppm)]
    
    hoomd.context.current.forces = forces
    _all = hoomd.group.all()
    hoomd.md.integrate.mode_standard(dt=0.0001)
    integrator = hoomd.md.integrate.nvt(group=_all, kT=2.0, tau=0.1)
    hoomd.dump.gsd("trajectories/start-shrink.gsd", period=None, group=_all, overwrite=True)
    hoomd.dump.gsd("trajectories/traj-shrink.gsd", period=1e5, group=_all, phase=0, overwrite=True)
    integrator.randomize_velocities(seed=42);

    hoomd.update.box_resize(L = hoomd.variant.linear_interp([(0,edge_nm*50), (1e6,edge_nm*10)], zero=0))
    hoomd.run(1e6)
    gsd_restart = hoomd.dump.gsd("trajectories/out-shrink.gsd", period=None, group=_all, overwrite=True)
    gsd_restart.write_restart()

/home/chris/miniconda3/envs/uli/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
build_peek_system(num_mols=20, poly_length=5, density=1.2)

/home/chris/miniconda3/envs/uli/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/chris/miniconda3/envs/uli/lib/python3.7/site-packages/mbuild/conversion.py:687: UserWarning: No unitcell detected for pybel.Molecule O=C1C=CC(=C[CH]1)C(=O)c1ccc(Oc2cc(Oc3ccc(C(=O)c4ccc(Oc5cc(Oc6ccc(C(=O)c7ccc(Oc8cc(Oc9ccc(C(=O)c%10ccc(Oc%11cc(Oc%12ccc(C(=O)c%13ccc(Oc%14ccccc%14)cc%13)cc%12)ccc%11)cc%10)cc9)ccc8)cc7)cc6)ccc5)cc4)cc3)ccc2)cc1	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))
/home/chris/miniconda3/envs/uli/lib/python3.7/site-packages/foyer/forcefield.py:449: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in fo

HOOMD-blue v2.9.2-15-g8c1194a45 CUDA (10.1) SINGLE SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX AVX2 
Compiled: 10/07/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
HOOMD-blue is running on the following GPU(s):
 [0]   GeForce GTX 1660 Ti  24 SM_7.5 @ 1.83 GHz, 5941 MiB DRAM, DIS, MNG


/home/chris/miniconda3/envs/uli/lib/python3.7/site-packages/foyer/forcefield.py:267: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 1900, Parameterized impropers: 1340. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)


notice(2): Group "all" created containing 3420 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 3 exclusions             : 1340
notice(2): Particles with 6 exclusions             : 180
notice(2): Particles with 7 exclusions             : 1340
notice(2): Particles with 8 exclusions             : 360
notice(2): Particles with 9 exclusions             : 200
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
Processing LJ and QQ
notice(2): Group "charged" created containing 0 particles
No charged groups found, ignoring electrostatics
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions
HOOMD SimulationContext updated from ParmEd Structure
** starting run **
Time 00:00:10 | Step 48660 / 1000000 | TPS 4865.96 | ETA 00:03:15
Time 00:00:20 | Step 97657 / 1000000 | TPS 4899.68 | ETA 00:03:04
Time 00:00:30

# -------------------------------------------------------------------------

In [ ]:
# Smiles Strings
'''
DEEPSMILES:
PEEK-para = "occccoccccC=O)cccccc6)))))))cc6)))))))cc6"
PEEK-meta = "occcoccccC=O)cccccc6)))))))cc6)))))))ccc6"
PEKK-para = "ccccoccccC=o)ccccC=o))cc6)))))))cc6)))))))cc6"
PEKK-meta = "ccccoccccC=o)cccC=o))ccc6)))))))cc6)))))))cc6"

SMILES:
PEEK-para = "oc3ccc(oc2ccc(C(=O)c1ccccc1)cc2)cc3"
PEEK-meta = "oc3cc(oc2ccc(C(=O)c1ccccc1)cc2)ccc3"
PEKK-para = "c3ccc(oc2ccc(C(=o)c1ccc(C=o)cc1)cc2)cc3"
PEKK-meta = "c3ccc(oc2ccc(C(=o)c1cc(C=o)ccc1)cc2)cc3"
'''


# DeepSMILES with bonded atom indicated
'''
PEEK Para Bonded:
"occccoccccC=O)cc*c*ccc6)))))))cc6)))))))cc6"
"occccoccccC=O)ccc*c*cc6)))))))cc6)))))))cc6"
"occccoccccC=O)cccc*c*c6)))))))cc6)))))))cc6"

PEEK Meta Bonded:
"occcoccccC=O)ccc*c*cc6)))))))cc6)))))))ccc6"
"occcoccccC=O)cc*c*ccc6)))))))cc6)))))))ccc6"
"occcoccccC=O)cccc*c*c6)))))))cc6)))))))ccc6"

PEKK Para Bonded
"ccccoccccC=o)cccc*C*=o))cc6)))))))cc6)))))))cc6"
PEKK Meta Bonded
"ccccoccccC=o)ccc*C*=o))ccc6)))))))cc6)))))))cc6"
'''


# Build up dicts and save to json file
'''
PEKK_dict = {}

PEKK_dict['para_deep_smiles'] = "ccccoccccC=o)ccccC=o))cc6)))))))cc6)))))))cc6"
PEKK_dict['meta_deep_smiles'] = "ccccoccccC=o)cccC=o))ccc6)))))))cc6)))))))cc6"
PEKK_dict['para_smiles'] = "c3ccc(oc2ccc(C(=o)c1ccc(C=o)cc1)cc2)cc3"
PEKK_dict['meta_smiles'] = "c3ccc(oc2ccc(C(=o)c1cc(C=o)ccc1)cc2)cc3"
PEKK_dict['para_template'] = "ccccoccccC=o)cccc*C*=o))cc6)))))))cc6)))))))cc6"
PEKK_dict['meta_template'] = "ccccoccccC=o)ccc*C*=o))ccc6)))))))cc6)))))))cc6"
PEKK_dict['monomer_mass'] = 302.329


PEEK_dict = {}

PEEK_dict['para_deep_smiles'] = "occccoccccC=O)cccccc6)))))))cc6)))))))cc6"
PEEK_dict['meta_deep_smiles'] = "occcoccccC=O)cccccc6)))))))cc6)))))))ccc6"
PEEK_dict['para_smiles'] = "oc3ccc(oc2ccc(C(=O)c1ccccc1)cc2)cc3"
PEEK_dict['meta_smiles'] = "oc3cc(oc2ccc(C(=O)c1ccccc1)cc2)ccc3"
PEEK_dict['para_template'] = "occccoccccC=O)ccc{}{}ccc6)))))))cc6)))))))cc6"
PEEK_dict['meta_template'] = "occcoccccC=O)cccc{}{}cc6)))))))cc6)))))))ccc6"
PEEK_dict['monomer_mass'] = 288.302



file_name = 'compounds/PEKK.json'
with open(file_name, 'w') as fp:
    json.dump(PEKK_dict, fp)
'''